# Euler-Lagrangian of a Triple-Pendulum on an Elastic Wheel

This is based on the `eulerlagrangian_cartpole` Notebook, but extended to a Triple-Pendulum. This way it models the Milana Robot. The other notebook is much simpler and better commented.

This notebook also takes into account the elasticity of the rubber in the wheels. That means the wheel can slightly move without changing its angle when the rubber is elastic enough. I added this because I thought it would explain some oscillations that occured while testing the robot but did not occur in the simulation. And it did so, but only with extreme pid gains and not in normal conditions. So it didn't end up very useful. But I still kept it in because it doesn't hurt.

In [1]:
from sympy import symbols, diff, simplify, Function
from sympy import sin, cos
import sympy

In [2]:
t = symbols('t')
gravity = symbols('g')
l1, l2, l3 = symbols('l1, l2, l3')
x1, x2, x3 = symbols('x1, x2, x3')
m1, m2, m3 = symbols('m1, m2, m3')
I1, I2, I3 = symbols('I1, I2, I3')
wheel_mass = symbols("m")
wheel_inertia = symbols("I_w")
rubber_inertia = symbols("I_b")
rubber_factor = symbols("b_f")
r = symbols('r')

wheel_pos = Function('w')(t)
wheel_vel = diff(wheel_pos, t)
wheel_acc = diff(wheel_vel, t)
rubber_pos = Function('ru')(t)
rubber_vel = diff(rubber_pos, t)
rubber_acc = diff(rubber_vel, t)
theta1 = Function('theta_1')(t)
theta1_vel = diff(theta1, t)
theta1_acc = diff(theta1_vel, t)
theta2 = Function('theta_2')(t)
theta2_vel = diff(theta2, t)
theta2_acc = diff(theta2_vel, t)
theta3 = Function('theta_3')(t)
theta3_vel = diff(theta3, t)
theta3_acc = diff(theta3_vel, t)

In [3]:
wheel_pos_ = symbols('q')
wheel_vel_ = symbols("\\dot{q}")
wheel_acc_ = symbols("\\ddot{q}")
rubber_pos_ = symbols('b')
rubber_vel_ = symbols("\\dot{b}")
rubber_acc_ = symbols("\\ddot{b}")
theta1_, theta2_, theta3_ = symbols("d_1, d_2, d_3")
theta1_vel_, theta2_vel_, theta3_vel_ = symbols("\\dot{d_1}, \\dot{d_2}, \\dot{d_3}")
theta1_acc_, theta2_acc_, theta3_acc_ = symbols("\\ddot{d_1}, \\ddot{d_2}, \\ddot{d_3}")
def remove_time(v):
    return v.subs([
        (rubber_acc, rubber_acc_),
        (rubber_vel, rubber_vel_),
        (rubber_pos, rubber_pos_),
        (wheel_acc, wheel_acc_),
        (wheel_vel, wheel_vel_),
        (wheel_pos, wheel_pos_),
        (theta1_acc, theta1_acc_),
        (theta2_acc, theta2_acc_),
        (theta3_acc, theta3_acc_),
        (theta1_vel, theta1_vel_),
        (theta2_vel, theta2_vel_),
        (theta3_vel, theta3_vel_),
        (theta1, theta1_),
        (theta2, theta2_),
        (theta3, theta3_),
    ])

In [4]:
x = theta1 + theta2 + theta1_vel + theta2_vel + theta2_acc + theta1_acc + wheel_pos + wheel_vel + wheel_acc + rubber_vel
x

theta_1(t) + theta_2(t) + w(t) + Derivative(ru(t), t) + Derivative(theta_1(t), t) + Derivative(theta_1(t), (t, 2)) + Derivative(theta_2(t), t) + Derivative(theta_2(t), (t, 2)) + Derivative(w(t), t) + Derivative(w(t), (t, 2))

In [5]:
remove_time(x)

\ddot{d_1} + \ddot{d_2} + \ddot{q} + \dot{b} + \dot{d_1} + \dot{d_2} + \dot{q} + d_1 + d_2 + q

In [6]:
def sq(v):
    return v**2
def lagrange():
    
    th2 = theta2
    th3 = theta3
    th2_vel = theta2_vel
    th3_vel = theta3_vel
        
    kinE = 0;
    kinE += wheel_mass * sq(rubber_vel*r)
    kinE += m1 * (sq(rubber_vel*r + x1*theta1_vel*cos(theta1)                                            ) + sq(x1*theta1_vel*sin(theta1)                                            ))
    kinE += m2 * (sq(rubber_vel*r + l1*theta1_vel*cos(theta1) + x2*th2_vel*cos(th2)                      ) + sq(l1*theta1_vel*sin(theta1) + x2*th2_vel*sin(th2)                      ))
    kinE += m3 * (sq(rubber_vel*r + l1*theta1_vel*cos(theta1) + l2*th2_vel*cos(th2) + x3*th3_vel*cos(th3)) + sq(l1*theta1_vel*sin(theta1) + l2*th2_vel*sin(th2) + x3*th3_vel*sin(th3)))
    kinE += rubber_inertia * sq(rubber_vel)
    kinE += wheel_inertia * sq(wheel_vel)
    kinE += I1 * sq(theta1_vel)
    kinE += I2 * sq(theta2_vel)
    kinE += I3 * sq(theta3_vel)
    kinE /= 2
    potE = gravity * (m1*x1*cos(theta1) + m2*(l1*cos(theta1)+x2*cos(th2)) + m3*(l1*cos(theta1)+l2*cos(th2)+x3*cos(th3)));
    potE += sq(rubber_pos-wheel_pos)*rubber_factor/2

    return kinE-potE

In [7]:
remove_time(lagrange())

I1*\dot{d_1}**2/2 + I2*\dot{d_2}**2/2 + I3*\dot{d_3}**2/2 + I_b*\dot{b}**2/2 + I_w*\dot{q}**2/2 + \dot{b}**2*m*r**2/2 - b_f*(b - q)**2/2 - g*(m1*x1*cos(d_1) + m2*(l1*cos(d_1) + x2*cos(d_2)) + m3*(l1*cos(d_1) + l2*cos(d_2) + x3*cos(d_3))) + m1*(\dot{d_1}**2*x1**2*sin(d_1)**2 + (\dot{b}*r + \dot{d_1}*x1*cos(d_1))**2)/2 + m2*((\dot{d_1}*l1*sin(d_1) + \dot{d_2}*x2*sin(d_2))**2 + (\dot{b}*r + \dot{d_1}*l1*cos(d_1) + \dot{d_2}*x2*cos(d_2))**2)/2 + m3*((\dot{d_1}*l1*sin(d_1) + \dot{d_2}*l2*sin(d_2) + \dot{d_3}*x3*sin(d_3))**2 + (\dot{b}*r + \dot{d_1}*l1*cos(d_1) + \dot{d_2}*l2*cos(d_2) + \dot{d_3}*x3*cos(d_3))**2)/2

In [8]:
e0 = remove_time(simplify(diff(diff(lagrange(), rubber_vel), t) - diff(lagrange(), rubber_pos)))
e0

I_b*\ddot{b} + \ddot{b}*m*r**2 + b_f*(b - q) + m1*r*(\ddot{b}*r + \ddot{d_1}*x1*cos(d_1) - \dot{d_1}**2*x1*sin(d_1)) + m2*r*(\ddot{b}*r + \ddot{d_1}*l1*cos(d_1) + \ddot{d_2}*x2*cos(d_2) - \dot{d_1}**2*l1*sin(d_1) - \dot{d_2}**2*x2*sin(d_2)) + m3*r*(\ddot{b}*r + \ddot{d_1}*l1*cos(d_1) + \ddot{d_2}*l2*cos(d_2) + \ddot{d_3}*x3*cos(d_3) - \dot{d_1}**2*l1*sin(d_1) - \dot{d_2}**2*l2*sin(d_2) - \dot{d_3}**2*x3*sin(d_3))

In [9]:
e1 = remove_time(simplify(diff(diff(lagrange(), wheel_vel), t) - diff(lagrange(), wheel_pos)))
e1

I_w*\ddot{q} - b_f*(b - q)

In [10]:
e2 = remove_time(simplify(diff(diff(lagrange(), theta1_vel), t) - diff(lagrange(), theta1)))
e2

I1*\ddot{d_1} + \ddot{b}*l1*m2*r*cos(d_1) + \ddot{b}*l1*m3*r*cos(d_1) + \ddot{b}*m1*r*x1*cos(d_1) + \ddot{d_1}*l1**2*m2 + \ddot{d_1}*l1**2*m3 + \ddot{d_1}*m1*x1**2 + \ddot{d_2}*l1*l2*m3*cos(d_1 - d_2) + \ddot{d_2}*l1*m2*x2*cos(d_1 - d_2) + \ddot{d_3}*l1*m3*x3*cos(d_1 - d_3) + \dot{d_2}**2*l1*l2*m3*sin(d_1 - d_2) + \dot{d_2}**2*l1*m2*x2*sin(d_1 - d_2) + \dot{d_3}**2*l1*m3*x3*sin(d_1 - d_3) - g*l1*m2*sin(d_1) - g*l1*m3*sin(d_1) - g*m1*x1*sin(d_1)

In [11]:
e3 = remove_time(simplify(diff(diff(lagrange(), theta2_vel), t) - diff(lagrange(), theta2)))
e3

I2*\ddot{d_2} + \ddot{b}*l2*m3*r*cos(d_2) + \ddot{b}*m2*r*x2*cos(d_2) + \ddot{d_1}*l1*l2*m3*cos(d_1 - d_2) + \ddot{d_1}*l1*m2*x2*cos(d_1 - d_2) + \ddot{d_2}*l2**2*m3 + \ddot{d_2}*m2*x2**2 + \ddot{d_3}*l2*m3*x3*cos(d_2 - d_3) - \dot{d_1}**2*l1*l2*m3*sin(d_1 - d_2) - \dot{d_1}**2*l1*m2*x2*sin(d_1 - d_2) + \dot{d_3}**2*l2*m3*x3*sin(d_2 - d_3) - g*l2*m3*sin(d_2) - g*m2*x2*sin(d_2)

In [12]:
e4 = remove_time(simplify(diff(diff(lagrange(), theta3_vel), t) - diff(lagrange(), theta3)))
e4

I3*\ddot{d_3} + \ddot{b}*m3*r*x3*cos(d_3) + \ddot{d_1}*l1*m3*x3*cos(d_1 - d_3) + \ddot{d_2}*l2*m3*x3*cos(d_2 - d_3) + \ddot{d_3}*m3*x3**2 - \dot{d_1}**2*l1*m3*x3*sin(d_1 - d_3) - \dot{d_2}**2*l2*m3*x3*sin(d_2 - d_3) - g*m3*x3*sin(d_3)

In [13]:
#a, b = sympy.linear_eq_to_matrix([e2, e3], [remove_time(x) for x in [theta1_acc, theta2_acc]])
#a, b = sympy.linear_eq_to_matrix([e1, e2, e3], [remove_time(x) for x in [cart_acc, theta1_acc, theta2_acc]])
#a, b = sympy.linear_eq_to_matrix([e2, e3, e4], [remove_time(x) for x in [theta1_acc, theta2_acc, theta3_acc]])
#a, b = sympy.linear_eq_to_matrix([e1, e2, e3, e4], [remove_time(x) for x in [wheel_acc, theta1_acc, theta2_acc, theta3_acc]])
a, b = sympy.linear_eq_to_matrix([e0, e1, e2, e3, e4], [remove_time(x) for x in [rubber_acc, wheel_acc, theta1_acc, theta2_acc, theta3_acc]])

In [14]:
simplify(a)

Matrix([
[I_b + m*r**2 + m1*r**2 + m2*r**2 + m3*r**2,   0,  r*(l1*m2 + l1*m3 + m1*x1)*cos(d_1),        r*(l2*m3 + m2*x2)*cos(d_2),        m3*r*x3*cos(d_3)],
[                                         0, I_w,                                   0,                                 0,                       0],
[        r*(l1*m2 + l1*m3 + m1*x1)*cos(d_1),   0, I1 + l1**2*m2 + l1**2*m3 + m1*x1**2, l1*(l2*m3 + m2*x2)*cos(d_1 - d_2), l1*m3*x3*cos(d_1 - d_3)],
[                r*(l2*m3 + m2*x2)*cos(d_2),   0,   l1*(l2*m3 + m2*x2)*cos(d_1 - d_2),          I2 + l2**2*m3 + m2*x2**2, l2*m3*x3*cos(d_2 - d_3)],
[                          m3*r*x3*cos(d_3),   0,             l1*m3*x3*cos(d_1 - d_3),           l2*m3*x3*cos(d_2 - d_3),           I3 + m3*x3**2]])

In [15]:
simplify(b)

Matrix([
[\dot{d_1}**2*m1*r*x1*sin(d_1) - b_f*(b - q) + m2*r*(\dot{d_1}**2*l1*sin(d_1) + \dot{d_2}**2*x2*sin(d_2)) + m3*r*(\dot{d_1}**2*l1*sin(d_1) + \dot{d_2}**2*l2*sin(d_2) + \dot{d_3}**2*x3*sin(d_3))],
[                                                                                                                                                                                     b_f*(b - q)],
[                    -\dot{d_2}**2*l1*l2*m3*sin(d_1 - d_2) - \dot{d_2}**2*l1*m2*x2*sin(d_1 - d_2) - \dot{d_3}**2*l1*m3*x3*sin(d_1 - d_3) + g*l1*m2*sin(d_1) + g*l1*m3*sin(d_1) + g*m1*x1*sin(d_1)],
[                                        \dot{d_1}**2*l1*l2*m3*sin(d_1 - d_2) + \dot{d_1}**2*l1*m2*x2*sin(d_1 - d_2) - \dot{d_3}**2*l2*m3*x3*sin(d_2 - d_3) + g*l2*m3*sin(d_2) + g*m2*x2*sin(d_2)],
[                                                                                                            m3*x3*(\dot{d_1}**2*l1*sin(d_1 - d_3) + \dot{d_2}**2*l2*sin(d_2 - d_3) + g*sin(d_3))]])

In [16]:
#simplify(a.det())